In [28]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling
import numpy as np

In [39]:
# Example usage:
input_raster_path = 'out_raster/classify_green.tif'
output_raster_path = 'out_raster/classify_green_resample_cubic_10.tif'
target_resolution = 10  # Set the desired target resolution

with rasterio.open(input_raster_path) as src:
        # Read the raster data
    raster_data = src.read(1)

        # Get the metadata of the input raster
    meta = src.meta.copy()

    # Calculate the new dimensions based on the target resolution
    new_height = int(src.height * src.res[0] / target_resolution)
    new_width = int(src.width * src.res[1] / target_resolution)

        # Perform spatial interpolation using bilinear resampling
    interpolated_data = np.empty((new_height, new_width), dtype=raster_data.dtype)
    rasterio.warp.reproject(
            source=raster_data,
            destination=interpolated_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=src.transform * src.transform.scale(
                (src.width / new_width), (src.height / new_height)
            ),
            dst_crs=src.crs,
            resampling=Resampling.bilinear
    )

        # Update metadata with new dimensions and resolution
    meta.update({
            'width': new_width,
            'height': new_height,
            'transform': src.transform * src.transform.scale(
                (src.width / new_width), (src.height / new_height)
            ),
            'res': (target_resolution, target_resolution)
        })



In [40]:
        # Write the interpolated raster to the output file
with rasterio.open(output_raster_path, 'w', **meta) as dst:
    dst.write(interpolated_data, 1)